In [3]:
import os
import json

import torch

import numpy as np
import pandas as pd

from torch import nn

# Setup

In [4]:
data_dir = os.path.join(os.curdir, "data")
vocab_path = os.path.join(data_dir, "word-level-vocab.json")
dataset_path = os.path.join(data_dir, "clean-tweets.tsv")

with open(vocab_path, "rt") as f:
    vocab = json.load(f)
    
dataset = pd.read_csv(filepath_or_buffer=dataset_path, sep="\t")

In [5]:
tweets = dataset["clean_text"].tolist()

In [6]:
OOV_TOKEN = "[OOV]"
PAD_TOKEN = "[PAD]"

OOV_INDEX = vocab.get(OOV_TOKEN)

print(f"Vocab Size = {len(vocab)}")

Vocab Size = 10998


In [35]:
tokenized_tweets = [[vocab.get(token) for token in tweet.split(" ") if token in vocab] for tweet in tweets]

In [30]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [31]:
tokenized_tweets = [torch.tensor(tweet) for tweet in tokenized_tweets]
tokenized_tweets = nn.utils.rnn.pad_sequence(tokenized_tweets, batch_first=True)